# Lets do RL in a Custom Environment created using Open Ai Gym

## Import Dependencies

In [24]:
import gymnasium as gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

## types of spaces avalable in gym

In [25]:
Discrete(3).sample()

1

In [26]:
Box(0,1,shape=(3,3)).sample()

array([[0.9531608 , 0.90417665, 0.5649612 ],
       [0.6138739 , 0.11115106, 0.99832445],
       [0.95112574, 0.5133277 , 0.4084161 ]], dtype=float32)

In [27]:
Box(0,255,shape=(3,3), dtype=int).sample() # to represent a tensor commonly used

array([[172,  20,  39],
       [ 83,  51,  58],
       [150,  39,  55]])

In [28]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([6.229503], dtype=float32))

In [29]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample() # created a dictionary of multiple spaces

OrderedDict([('height', 0), ('speed', array([90.53036], dtype=float32))])

In [30]:
MultiBinary(4).sample()

array([1, 1, 1, 0], dtype=int8)

In [31]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 1])

## Building an environment

### Build  a shower environment with temperature going to fluctuate between 37 and 39 degrees

### then we build an agent to give us a best shower

In [32]:
class ShowerEnv(gym.Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3) # 3 actions can take
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100])) #to store the observation state+action+reward
        # Set start temp
        self.state = 38 + random.randint(-3,3) # temeperature between 37 and 39
        # Set shower length
        self.shower_length = 60 #episode length
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature reduce
        # 1 -1 = 0 no change in temperature
        # 2 -1 = 1 temperature increase
        self.state += action -1 
        
        # Reduce shower length by 1 second after every action
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 # for comfortable temperature
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        #to visualize
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [33]:
env=ShowerEnv()

In [34]:
env.observation_space.sample()

array([73.12013], dtype=float32)

In [35]:
env.reset()

array([35.])

In [36]:
from stable_baselines3.common.env_checker import check_env

In [38]:
# check_env(env, warn=True)

# Lets test the Environment

In [39]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
# env.close()

Episode:1 Score:46
Episode:2 Score:36
Episode:3 Score:-54
Episode:4 Score:-10
Episode:5 Score:-22


In [ ]:
env.close()

# Lets train the model or the agent using our environment and policy

In [40]:
log_path=os.path.join('pytorch_RL','logs')

In [42]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path) #seting up the agent using multilayerpAgency

In [ ]:
model.learn(total_timesteps=400000)

# Lets save the model

In [ ]:
model.save(shower_path)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)